In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pywt

In [2]:
df = pd.read_excel("df_100.xlsx")
df.head()

,AvgAir_T_TempC,MaxAir_T_TempC,MinAir_T_TempC,Avg_Soil_TP100_TempC,Max_Soil_TP100_TempC,Min_Soil_TP100_TempC
0,24.24,30.42,0.00,10.860,8.910,7.823
1,22.25,28.47,16.63,8.150,8.910,7.823
2,20.45,24.77,16.77,7.739,7.823,7.702
3,20.11,25.32,14.59,7.795,7.890,7.717
4,23.52,28.47,17.78,7.982,8.080,7.880


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AvgAir_T_TempC        433 non-null    float64
 1   MaxAir_T_TempC        433 non-null    float64
 2   MinAir_T_TempC        433 non-null    float64
 3   Avg_Soil_TP100_TempC  433 non-null    float64
 4   Max_Soil_TP100_TempC  433 non-null    float64
 5   Min_Soil_TP100_TempC  433 non-null    float64
dtypes: float64(6)
memory usage: 20.4 KB


In [4]:
df["Avg_Soil_TP100_TempC_Plus_1"] = df["Avg_Soil_TP100_TempC"].shift(-1)
df["Avg_Soil_TP100_TempC_Plus_2"] = df["Avg_Soil_TP100_TempC"].shift(-2)

df["Max_Soil_TP100_TempC_Plus_1"] = df["Max_Soil_TP100_TempC"].shift(-1)
df["Max_Soil_TP100_TempC_Plus_2"] = df["Max_Soil_TP100_TempC"].shift(-2)

df["Min_Soil_TP100_TempC_Plus_1"] = df["Min_Soil_TP100_TempC"].shift(-1)
df["Min_Soil_TP100_TempC_Plus_2"] = df["Min_Soil_TP100_TempC"].shift(-2)

df["AvgAir_T_TempC_Minus_6"] = df["AvgAir_T_TempC"].shift(6)
df["AvgAir_T_TempC_Minus_5"] = df["AvgAir_T_TempC"].shift(5)
df["AvgAir_T_TempC_Minus_4"] = df["AvgAir_T_TempC"].shift(4)
df["AvgAir_T_TempC_Minus_3"] = df["AvgAir_T_TempC"].shift(3)
df["AvgAir_T_TempC_Minus_2"] = df["AvgAir_T_TempC"].shift(2)
df["AvgAir_T_TempC_Minus_1"] = df["AvgAir_T_TempC"].shift(1)

df["MaxAir_T_TempC_Minus_6"] = df["MaxAir_T_TempC"].shift(6)
df["MaxAir_T_TempC_Minus_5"] = df["MaxAir_T_TempC"].shift(5)
df["MaxAir_T_TempC_Minus_4"] = df["MaxAir_T_TempC"].shift(4)
df["MaxAir_T_TempC_Minus_3"] = df["MaxAir_T_TempC"].shift(3)
df["MaxAir_T_TempC_Minus_2"] = df["MaxAir_T_TempC"].shift(2)
df["MaxAir_T_TempC_Minus_1"] = df["MaxAir_T_TempC"].shift(1)


df["MinAir_T_TempC_Minus_6"] = df["MinAir_T_TempC"].shift(6)
df["MinAir_T_TempC_Minus_5"] = df["MinAir_T_TempC"].shift(5)
df["MinAir_T_TempC_Minus_4"] = df["MinAir_T_TempC"].shift(4)
df["MinAir_T_TempC_Minus_3"] = df["MinAir_T_TempC"].shift(3)
df["MinAir_T_TempC_Minus_2"] = df["MinAir_T_TempC"].shift(2)
df["MinAir_T_TempC_Minus_1"] = df["MinAir_T_TempC"].shift(1)

In [5]:
df = df[["AvgAir_T_TempC_Minus_6","MaxAir_T_TempC_Minus_6","MinAir_T_TempC_Minus_6",
          "AvgAir_T_TempC_Minus_5","MaxAir_T_TempC_Minus_5","MinAir_T_TempC_Minus_5",
          "AvgAir_T_TempC_Minus_4","MaxAir_T_TempC_Minus_4","MinAir_T_TempC_Minus_4",
          "AvgAir_T_TempC_Minus_3","MaxAir_T_TempC_Minus_3","MinAir_T_TempC_Minus_3",
          "AvgAir_T_TempC_Minus_2","MaxAir_T_TempC_Minus_2","MinAir_T_TempC_Minus_2",
          "AvgAir_T_TempC_Minus_1","MaxAir_T_TempC_Minus_1","MinAir_T_TempC_Minus_1",
          "AvgAir_T_TempC","MaxAir_T_TempC","MinAir_T_TempC",
          "Avg_Soil_TP100_TempC","Max_Soil_TP100_TempC","Min_Soil_TP100_TempC",
          "Avg_Soil_TP100_TempC_Plus_1","Max_Soil_TP100_TempC_Plus_1","Min_Soil_TP100_TempC_Plus_1",
          "Avg_Soil_TP100_TempC_Plus_2","Max_Soil_TP100_TempC_Plus_2","Min_Soil_TP100_TempC_Plus_2"]]
df.dropna(inplace=True)

In [6]:
df.reset_index(drop="True", inplace=True)

In [7]:
def WaveletTransform(x,walvelettype,level):
    
    size = np.shape(x)
    length = (size[0] // (2**level)) * (2 ** level)
    x = x.iloc[0:length]
    bigmats = []
    for i in x.columns:
        vec = x[i].values
        cAs_cDs = pywt.swt(vec, wavelet=walvelettype, level=level,axis=0) 
        for j in cAs_cDs:
            bigmats.append(j[0])
            bigmats.append(j[1])

    
    features_num = len(bigmats)
    x = np.zeros((length, features_num))
    for i in range(0 ,len(bigmats)):
        for j in range(0,len(bigmats[0])):
            x[j,i] = x[j,i] + bigmats[i][j]
    
    return(x)

In [8]:
x = df.drop(["Avg_Soil_TP100_TempC","Max_Soil_TP100_TempC","Min_Soil_TP100_TempC",
            "Avg_Soil_TP100_TempC_Plus_1","Max_Soil_TP100_TempC_Plus_1","Min_Soil_TP100_TempC_Plus_1",
            "Avg_Soil_TP100_TempC_Plus_2","Max_Soil_TP100_TempC_Plus_2","Min_Soil_TP100_TempC_Plus_2"],axis=1)

In [9]:
y = df[["Avg_Soil_TP100_TempC","Max_Soil_TP100_TempC","Min_Soil_TP100_TempC",
            "Avg_Soil_TP100_TempC_Plus_1","Max_Soil_TP100_TempC_Plus_1","Min_Soil_TP100_TempC_Plus_1",
            "Avg_Soil_TP100_TempC_Plus_2","Max_Soil_TP100_TempC_Plus_2","Min_Soil_TP100_TempC_Plus_2"]].values

In [10]:
x = WaveletTransform(x,"db5",5)

In [11]:
x.shape

(416, 210)

In [12]:
y = df[0:x.shape[0]]

In [13]:
x = pd.DataFrame(x)

In [14]:
df = pd.concat([x,y],axis=1)

In [15]:
df

,0,1,2,3,4,5,6,7,8,9,...,MinAir_T_TempC,Avg_Soil_TP100_TempC,Max_Soil_TP100_TempC,Min_Soil_TP100_TempC,Avg_Soil_TP100_TempC_Plus_1,Max_Soil_TP100_TempC_Plus_1,Min_Soil_TP100_TempC_Plus_1,Avg_Soil_TP100_TempC_Plus_2,Max_Soil_TP100_TempC_Plus_2,Min_Soil_TP100_TempC_Plus_2
0,63.788969,-2.934640,81.092130,0.699315,54.865099,-4.801267,46.136754,0.933353,23.939074,2.294474,...,11.12,8.37,8.47,8.27,8.56,8.73,8.46,8.88,9.02,8.72
1,65.274606,-4.381344,81.312692,2.015804,55.739448,-5.276362,42.107493,1.293356,25.742177,-1.822694,...,15.37,8.56,8.73,8.46,8.88,9.02,8.72,9.14,9.26,9.01
2,66.787133,-5.580971,81.369152,3.158354,57.132431,-4.032281,37.995890,0.213967,30.462259,-0.640552,...,14.90,8.88,9.02,8.72,9.14,9.26,9.01,9.37,9.47,9.25
3,68.318966,-6.573750,81.301376,3.775352,58.577150,-2.020705,35.516697,-2.364010,32.229497,1.145901,...,12.12,9.14,9.26,9.01,9.37,9.47,9.25,9.57,9.67,9.47
4,69.863607,-7.439896,81.153640,3.698452,59.803503,0.418576,34.832882,-1.677906,29.836496,-0.260397,...,12.12,9.37,9.47,9.25,9.57,9.67,9.47,9.73,9.81,9.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,56.139065,4.532026,78.420099,-2.731537,55.563476,3.619056,38.098491,2.361043,32.114807,-1.926774,...,10.62,13.23,13.24,13.21,13.23,13.24,13.20,13.21,13.23,13.18
412,57.739675,3.264100,79.060169,-2.864572,54.922004,2.698650,41.865780,-3.203784,29.541111,3.124138,...,6.37,13.23,13.24,13.20,13.21,13.23,13.18,13.17,13.20,13.15
413,59.298596,1.813591,79.677525,-2.274863,54.424450,1.157306,46.086753,-4.512755,26.527360,-1.981640,...,13.19,13.21,13.23,13.18,13.17,13.20,13.15,13.14,13.17,13.12
414,60.814507,0.257546,80.231815,-1.368339,54.232844,-0.960877,48.590673,-0.748511,24.454810,-1.022841,...,9.16,13.17,13.20,13.15,13.14,13.17,13.12,13.12,13.14,13.10


# 2 - Feature Selection

In [16]:
# df --> Dataframe
#Ys --> a list of all target variables
#y --> the main target variable - string
def FeatureSelection(df,Ys,y,corr):
    X = df.drop(Ys,axis=1)
    Y = df[y]
    df_final = pd.concat([X,Y],axis=1)
    indexes = df_final.corr()[df_final.corr()[y] > corr].index
    df_final = df_final[indexes]
    return df_final

In [17]:
Ys = ["Avg_Soil_TP100_TempC", "Max_Soil_TP100_TempC", "Min_Soil_TP100_TempC","Avg_Soil_TP100_TempC_Plus_1",
     "Max_Soil_TP100_TempC_Plus_1","Min_Soil_TP100_TempC_Plus_1", "Avg_Soil_TP100_TempC_Plus_2", 
      "Max_Soil_TP100_TempC_Plus_2", "Min_Soil_TP100_TempC_Plus_2"]

In [18]:
df_avg = FeatureSelection(df,Ys,y = "Avg_Soil_TP100_TempC",corr=0.5)
df_max = FeatureSelection(df,Ys,y = "Max_Soil_TP100_TempC",corr=0.5)
df_min = FeatureSelection(df,Ys,y = "Min_Soil_TP100_TempC",corr=0.5)
df_avg_plus1 = FeatureSelection(df,Ys,y = "Avg_Soil_TP100_TempC_Plus_1",corr=0.5)
df_max_plus1 = FeatureSelection(df,Ys,y = "Max_Soil_TP100_TempC_Plus_1",corr=0.5)
df_min_plus1 = FeatureSelection(df,Ys,y = "Min_Soil_TP100_TempC_Plus_1",corr=0.5)
df_avg_plus2 = FeatureSelection(df,Ys,y = "Avg_Soil_TP100_TempC_Plus_2",corr=0.5)
df_max_plus2 = FeatureSelection(df,Ys,y = "Max_Soil_TP100_TempC_Plus_2",corr=0.5)
df_min_plus2 = FeatureSelection(df,Ys,y = "Min_Soil_TP100_TempC_Plus_2",corr=0.5)

# 3 - SVR Support vector machines for regression Problem

### Train test split and data transform

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [20]:
#df: data frame
#y: target variable --> string
def SplitTransform(df,y,test_size, scaler):
    X = df.drop(y,axis=1).values
    y = df[y].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=101)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    data = [X_train, X_test,y_train,y_test]
    return data

In [21]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score

In [22]:
model = SVR(gamma="auto")

### Avg_Soil_TP100_TempC

In [23]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [24]:
Ys = {"Avg_Soil_TP100_TempC":df_avg, "Max_Soil_TP100_TempC":df_max, "Min_Soil_TP100_TempC":df_min,"Avg_Soil_TP100_TempC_Plus_1":df_avg_plus1,
     "Max_Soil_TP100_TempC_Plus_1":df_max_plus1,"Min_Soil_TP100_TempC_Plus_1":df_min_plus1, "Avg_Soil_TP100_TempC_Plus_2":df_avg_plus2, 
      "Max_Soil_TP100_TempC_Plus_2":df_max_plus2, "Min_Soil_TP100_TempC_Plus_2":df_min_plus2}

In [25]:
i = "Avg_Soil_TP100_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Avg_Soil_TP100_TempC',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9915900793521116,
 'MAE': 0.19550608812381784,
 'MSE': 0.1781730421458661}

### Max_Soil_TP100_TempC

In [26]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [27]:
i = "Max_Soil_TP100_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Max_Soil_TP100_TempC',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9918666267820498,
 'MAE': 0.19417501090127548,
 'MSE': 0.17308280299684423}

### Min_Soil_TP100_TempC

In [28]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [29]:
i = "Min_Soil_TP100_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Min_Soil_TP100_TempC',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9910928570470595,
 'MAE': 0.19863843335963513,
 'MSE': 0.1879560209015293}

### Avg_Soil_TP100_TempC_Plus_1

In [30]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [31]:
i = "Avg_Soil_TP100_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Avg_Soil_TP100_TempC_Plus_1',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9924623519783546,
 'MAE': 0.1892703313708754,
 'MSE': 0.16082353751262785}

### Max_Soil_TP100_TempC_Plus_1

In [32]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [33]:
i = "Max_Soil_TP100_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Max_Soil_TP100_TempC_Plus_1',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9928992540772976,
 'MAE': 0.1849756632692827,
 'MSE': 0.1521308478265335}

### Min_Soil_TP100_TempC_Plus_1

In [34]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [35]:
i = "Min_Soil_TP100_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Min_Soil_TP100_TempC_Plus_1',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9920840992922342,
 'MAE': 0.19045070542523093,
 'MSE': 0.16826862758445135}

### Avg_Soil_TP100_TempC_Plus_2

In [36]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [37]:
i = "Avg_Soil_TP100_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Avg_Soil_TP100_TempC_Plus_2',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9933976190169944,
 'MAE': 0.1771505581070886,
 'MSE': 0.14185741750864386}

### Max_Soil_TP100_TempC_Plus_2

In [38]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [39]:
i = "Max_Soil_TP100_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Max_Soil_TP100_TempC_Plus_2',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9936390138725153,
 'MAE': 0.17604772435556076,
 'MSE': 0.13721335666006873}

### Min_Soil_TP100_TempC_Plus_2

In [40]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [41]:
i = "Min_Soil_TP100_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Min_Soil_TP100_TempC_Plus_2',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9930390368436273,
 'MAE': 0.17915475545497128,
 'MSE': 0.1491094949173805}

# 4 - XGBoost (Extreme Gradient Boosting Method) for regression problem

### Building the model

In [42]:
import xgboost as xg

In [43]:
model = xg.XGBRegressor(objective ='reg:squarederror') 

### Avg_Soil_TP100_TempC

In [44]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [0],
    'reg_lambda': [1.0],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [45]:
i = "Avg_Soil_TP100_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.996, total=   0.5s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.995, total=   0.5s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.996, total=   0.5s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.985, total=   0.4s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.992, total=   0.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.3s finished


{'Target Variable': 'Avg_Soil_TP100_TempC',
 'Best Parameters': {'gamma': 0,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1.0},
 'R Squared': 0.996829329438783,
 'MAE': 0.15882168276977546,
 'MSE': 0.06717400118111867}

### Max_Soil_TP100_TempC

In [46]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [0],
    'reg_lambda': [1.0],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [47]:
i = "Max_Soil_TP100_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.996, total=   0.6s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.996, total=   0.6s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.997, total=   0.6s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.986, total=   0.6s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.990, total=   0.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.7s finished


{'Target Variable': 'Max_Soil_TP100_TempC',
 'Best Parameters': {'gamma': 0,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1.0},
 'R Squared': 0.9970315180659882,
 'MAE': 0.14860117778205864,
 'MSE': 0.06317098207792983}

### Min_Soil_TP100_TempC

In [48]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [0.25],
    'reg_lambda': [1.0],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [49]:
i = "Min_Soil_TP100_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.996, total=   0.5s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.994, total=   0.6s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.994, total=   0.6s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.986, total=   0.6s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.989, total=   0.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.7s finished


{'Target Variable': 'Min_Soil_TP100_TempC',
 'Best Parameters': {'gamma': 0.25,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1.0},
 'R Squared': 0.9959155745952011,
 'MAE': 0.1825060631389618,
 'MSE': 0.08618839405756863}

### Avg_Soil_TP100_TempC_Plus_1

In [50]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [0],
    'reg_lambda': [1],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [51]:
i = "Avg_Soil_TP100_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.996, total=   1.2s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.995, total=   0.7s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.996, total=   0.7s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.988, total=   0.9s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.990, total=   1.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.5s finished


{'Target Variable': 'Avg_Soil_TP100_TempC_Plus_1',
 'Best Parameters': {'gamma': 0,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1},
 'R Squared': 0.9965024839268108,
 'MAE': 0.16797815191078183,
 'MSE': 0.07462313254510146}

### Max_Soil_TP100_TempC_Plus_1

In [52]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [1],
    'reg_lambda': [1],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [53]:
i = "Max_Soil_TP100_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.995, total=   0.7s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.994, total=   0.5s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.993, total=   0.6s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.989, total=   0.7s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.8s finished


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.988, total=   0.4s


{'Target Variable': 'Max_Soil_TP100_TempC_Plus_1',
 'Best Parameters': {'gamma': 1,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1},
 'R Squared': 0.996271270275316,
 'MAE': 0.20508357627487173,
 'MSE': 0.07988665141764204}

### Min_Soil_TP100_TempC_Plus_1

In [54]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [0.25],
    'reg_lambda': [10],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [55]:
i = "Min_Soil_TP100_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=10 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=10, score=0.991, total=   0.5s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=10, score=0.993, total=   0.4s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=10 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=10, score=0.978, total=   0.8s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=10 
[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=10, score=0.977, total=   0.4s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=10 
[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=10, score=0.984, total=   0.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.8s finished


{'Target Variable': 'Min_Soil_TP100_TempC_Plus_1',
 'Best Parameters': {'gamma': 0.25,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 10},
 'R Squared': 0.9931372239457102,
 'MAE': 0.26389244345664975,
 'MSE': 0.14588231342289007}

### Avg_Soil_TP100_TempC_Plus_2

In [56]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [1],
    'reg_lambda': [1],
    'n_estimators' : [100]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [57]:
i = "Avg_Soil_TP100_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=100, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=100, reg_lambda=1, score=0.994, total=   0.3s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=100, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=100, reg_lambda=1, score=0.995, total=   0.3s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=100, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=100, reg_lambda=1, score=0.993, total=   0.3s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=100, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=100, reg_lambda=1, score=0.988, total=   0.3s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=100, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=100, reg_lambda=1, score=0.989, total=   0.3s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.5s finished


{'Target Variable': 'Avg_Soil_TP100_TempC_Plus_2',
 'Best Parameters': {'gamma': 1,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 100,
  'reg_lambda': 1},
 'R Squared': 0.9955690724154506,
 'MAE': 0.2144895813922882,
 'MSE': 0.09520201059737414}

### Max_Soil_TP100_TempC_Plus_2

In [58]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [1],
    'reg_lambda': [1],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [59]:
i = "Max_Soil_TP100_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.995, total=   0.5s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.995, total=   0.4s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.993, total=   0.5s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.990, total=   0.4s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.989, total=   0.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.2s finished


{'Target Variable': 'Max_Soil_TP100_TempC_Plus_2',
 'Best Parameters': {'gamma': 1,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1},
 'R Squared': 0.9960817306282319,
 'MAE': 0.21270037255668633,
 'MSE': 0.08452131195123892}

### Min_Soil_TP100_TempC_Plus_2

In [60]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [1],
    'reg_lambda': [1],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [61]:
i = "Min_Soil_TP100_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.994, total=   0.6s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.995, total=   0.6s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.994, total=   0.5s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.989, total=   0.4s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.988, total=   0.4s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.4s finished


{'Target Variable': 'Min_Soil_TP100_TempC_Plus_2',
 'Best Parameters': {'gamma': 1,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1},
 'R Squared': 0.9952073985535266,
 'MAE': 0.22633237481117238,
 'MSE': 0.10266142270408726}

# 5 - ANN

### Building the Model

In [62]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [63]:
def create_model(neurons,hidden_layers, optimizer, activation):
    model = Sequential()
    
    # Add aninput layer
    model.add(Dense(X_train.shape[1], activation = activation))
    
    for i in range(hidden_layers):
    # Add one hidden layer
        model.add(Dense(neurons, activation=activation))
        
        
    # Add an output layer 
    model.add(Dense(1))
    
    # Compile model
    model.compile(optimizer = optimizer, loss="mse")
    
    return model

In [64]:
model = KerasRegressor(build_fn=create_model, verbose = 3)

### Avg_Soil_TP100_TempC

In [65]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [350],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["relu"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1, verbose=3)

In [66]:
i = "Avg_Soil_TP100_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   31.3s remaining:   47.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   37.8s finished


Epoch 1/400
3/3 [==============================] - 0s 5ms/step - loss: 47.0179
Epoch 2/400
3/3 [==============================] - 0s 5ms/step - loss: 10.2134
Epoch 3/400
3/3 [==============================] - 0s 4ms/step - loss: 4.6499
Epoch 4/400
3/3 [==============================] - 0s 4ms/step - loss: 5.0880
Epoch 5/400
3/3 [==============================] - 0s 4ms/step - loss: 2.2383
Epoch 6/400
3/3 [==============================] - 0s 6ms/step - loss: 2.3435
Epoch 7/400
3/3 [==============================] - 0s 5ms/step - loss: 1.0030
Epoch 8/400
3/3 [==============================] - 0s 5ms/step - loss: 1.3084
Epoch 9/400
3/3 [==============================] - 0s 4ms/step - loss: 0.9398
Epoch 10/400
3/3 [==============================] - 0s 4ms/step - loss: 0.7720
Epoch 11/400
3/3 [==============================] - 0s 4ms/step - loss: 0.6659
Epoch 12/400
3/3 [==============================] - 0s 5ms/step - loss: 0.5550
Epoch 13/400
3/3 [==============================] - 0s 6ms/

3/3 [==============================] - 0s 4ms/step - loss: 0.0160
Epoch 105/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0120
Epoch 106/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0149
Epoch 107/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0108
Epoch 108/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0154
Epoch 109/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0138
Epoch 110/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0126
Epoch 111/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0097
Epoch 112/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0098
Epoch 113/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0096
Epoch 114/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0113
Epoch 115/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0101
Epoch 116/400
3/3 [==============================] - 0

3/3 [==============================] - 0s 5ms/step - loss: 0.0052
Epoch 206/400
3/3 [==============================] - ETA: 0s - loss: 0.002 - 0s 4ms/step - loss: 0.0038
Epoch 207/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0047
Epoch 208/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0059
Epoch 209/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0032
Epoch 210/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0023
Epoch 211/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0038
Epoch 212/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0032
Epoch 213/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0029
Epoch 214/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0017
Epoch 215/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0027
Epoch 216/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0022
Epoch 217/400
3/3 [===========

3/3 [==============================] - 0s 4ms/step - loss: 0.0053
Epoch 307/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0040
Epoch 308/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0043
Epoch 309/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0032
Epoch 310/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0080
Epoch 311/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0094
Epoch 312/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0070
Epoch 313/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0052
Epoch 314/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0036
Epoch 315/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0037
Epoch 316/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0051
Epoch 317/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0079
Epoch 318/400
3/3 [==============================] - 0

{'Target Variable': 'Avg_Soil_TP100_TempC',
 'Best Parameters': {'activation': 'relu',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 350,
  'optimizer': 'Adam'},
 'R Squared': 0.9993025983739133,
 'MAE': 0.09000138709259038,
 'MSE': 0.01517639131082489}

### Max_Soil_TP100_TempC

In [67]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [150],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [68]:
i = "Max_Soil_TP100_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
3/3 [==============================] - 0s 3ms/step - loss: 77.6797
Epoch 2/400
3/3 [==============================] - 0s 3ms/step - loss: 61.6220
Epoch 3/400
3/3 [==============================] - 0s 3ms/step - loss: 49.0574
Epoch 4/400
3/3 [==============================] - 0s 3ms/step - loss: 39.3018
Epoch 5/400
3/3 [==============================] - 0s 5ms/step - loss: 32.7803
Epoch 6/400
3/3 [==============================] - 0s 3ms/step - loss: 28.7544
Epoch 7/400
3/3 [==============================] - 0s 3ms/step - loss: 25.8733
Epoch 8/400
3/3 [==============================] - 0s 2ms/step - loss: 24.9556
Epoch 9/400
3/3 [==============================] - 0s 3ms/step - loss: 24.3057
Epoch 10/400
3/3 [==============================] - 0s 3ms/step - loss: 24.2382
Epoch 11/400
3/3 [==============================] - 0s 3ms/step - loss: 24.2438
Epoch 12/400
3/3 [==============================] - 0s 5ms/step - loss: 24.3838
Epoch 13/400
3/3 [==============================]

3/3 [==============================] - 0s 3ms/step - loss: 0.3815
Epoch 104/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3783
Epoch 105/400
3/3 [==============================] - 0s 2ms/step - loss: 0.3775
Epoch 106/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3765
Epoch 107/400
3/3 [==============================] - 0s 2ms/step - loss: 0.3750
Epoch 108/400
3/3 [==============================] - 0s 2ms/step - loss: 0.3735
Epoch 109/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3721
Epoch 110/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3719
Epoch 111/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3700
Epoch 112/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3696
Epoch 113/400
3/3 [==============================] - ETA: 0s - loss: 0.411 - 0s 2ms/step - loss: 0.3683
Epoch 114/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3681
Epoch 115/400
3/3 [===========

3/3 [==============================] - 0s 3ms/step - loss: 0.2920
Epoch 205/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2919
Epoch 206/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2939
Epoch 207/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2981
Epoch 208/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2923
Epoch 209/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3134
Epoch 210/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2959
Epoch 211/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2930
Epoch 212/400
3/3 [==============================] - 0s 2ms/step - loss: 0.3191
Epoch 213/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3385
Epoch 214/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2570
Epoch 215/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2906
Epoch 216/400
3/3 [==============================] - 0

3/3 [==============================] - 0s 3ms/step - loss: 0.0410
Epoch 306/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0522
Epoch 307/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0488
Epoch 308/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0420
Epoch 309/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0371
Epoch 310/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0376
Epoch 311/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0377
Epoch 312/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0396
Epoch 313/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0366
Epoch 314/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0356
Epoch 315/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0368
Epoch 316/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0344
Epoch 317/400
3/3 [==============================] - 0

{'Target Variable': 'Max_Soil_TP100_TempC',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 150,
  'optimizer': 'Adam'},
 'R Squared': 0.998564392418358,
 'MAE': 0.12269413408470155,
 'MSE': 0.03087351450919764}

### Min_Soil_TP100_TempC

In [69]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [150],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [70]:
i = "Min_Soil_TP100_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
3/3 [==============================] - 0s 3ms/step - loss: 79.2702
Epoch 2/400
3/3 [==============================] - 0s 3ms/step - loss: 62.5960
Epoch 3/400
3/3 [==============================] - 0s 3ms/step - loss: 49.3549
Epoch 4/400
3/3 [==============================] - 0s 3ms/step - loss: 39.3453
Epoch 5/400
3/3 [==============================] - 0s 5ms/step - loss: 32.9167
Epoch 6/400
3/3 [==============================] - 0s 3ms/step - loss: 28.6150
Epoch 7/400
3/3 [==============================] - 0s 3ms/step - loss: 26.2374
Epoch 8/400
3/3 [==============================] - 0s 2ms/step - loss: 24.9564
Epoch 9/400
3/3 [==============================] - 0s 3ms/step - loss: 24.3665
Epoch 10/400
3/3 [==============================] - 0s 2ms/step - loss: 24.2475
Epoch 11/400
3/3 [==============================] - 0s 3ms/step - loss: 24.1557
Epoch 12/400
3/3 [==============================] - 0s 4ms/step - loss: 24.2207
Epoch 13/400
3/3 [==============================]

3/3 [==============================] - 0s 4ms/step - loss: 0.4207
Epoch 104/400
3/3 [==============================] - 0s 3ms/step - loss: 0.4176
Epoch 105/400
3/3 [==============================] - 0s 3ms/step - loss: 0.4146
Epoch 106/400
3/3 [==============================] - 0s 2ms/step - loss: 0.4126
Epoch 107/400
3/3 [==============================] - 0s 3ms/step - loss: 0.4117
Epoch 108/400
3/3 [==============================] - 0s 2ms/step - loss: 0.4094
Epoch 109/400
3/3 [==============================] - 0s 2ms/step - loss: 0.4102
Epoch 110/400
3/3 [==============================] - 0s 3ms/step - loss: 0.4086
Epoch 111/400
3/3 [==============================] - 0s 3ms/step - loss: 0.4080
Epoch 112/400
3/3 [==============================] - 0s 3ms/step - loss: 0.4060
Epoch 113/400
3/3 [==============================] - 0s 3ms/step - loss: 0.4030
Epoch 114/400
3/3 [==============================] - 0s 3ms/step - loss: 0.4014
Epoch 115/400
3/3 [==============================] - 0

3/3 [==============================] - 0s 3ms/step - loss: 0.2766
Epoch 206/400
3/3 [==============================] - 0s 2ms/step - loss: 0.2888
Epoch 207/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2574
Epoch 208/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2620
Epoch 209/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2582
Epoch 210/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2465
Epoch 211/400
3/3 [==============================] - ETA: 0s - loss: 0.267 - 0s 2ms/step - loss: 0.2513
Epoch 212/400
3/3 [==============================] - 0s 2ms/step - loss: 0.2512
Epoch 213/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2545
Epoch 214/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2371
Epoch 215/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2485
Epoch 216/400
3/3 [==============================] - 0s 2ms/step - loss: 0.2347
Epoch 217/400
3/3 [===========

3/3 [==============================] - 0s 3ms/step - loss: 0.0393
Epoch 307/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0337
Epoch 308/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0333
Epoch 309/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0323
Epoch 310/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0309
Epoch 311/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0306
Epoch 312/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0322
Epoch 313/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0332
Epoch 314/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0325
Epoch 315/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0328
Epoch 316/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 317/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0320
Epoch 318/400
3/3 [==============================] - 0

{'Target Variable': 'Min_Soil_TP100_TempC',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 150,
  'optimizer': 'Adam'},
 'R Squared': 0.9984872471011118,
 'MAE': 0.1425116953496932,
 'MSE': 0.04115392156991526}

### Avg_Soil_TP100_TempC_Plus_1

In [71]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [350],
    'hidden_layers': [6],
    'optimizer' : ['Adam'],
    "activation" : ["relu"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [72]:
i = "Avg_Soil_TP100_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
3/3 [==============================] - 0s 7ms/step - loss: 60.6003
Epoch 2/400
3/3 [==============================] - 0s 6ms/step - loss: 24.4600
Epoch 3/400
3/3 [==============================] - 0s 6ms/step - loss: 5.7360
Epoch 4/400
3/3 [==============================] - 0s 8ms/step - loss: 5.6636
Epoch 5/400
3/3 [==============================] - 0s 6ms/step - loss: 2.0825
Epoch 6/400
3/3 [==============================] - 0s 6ms/step - loss: 1.3936
Epoch 7/400
3/3 [==============================] - 0s 7ms/step - loss: 2.2107
Epoch 8/400
3/3 [==============================] - 0s 13ms/step - loss: 0.9709
Epoch 9/400
3/3 [==============================] - 0s 9ms/step - loss: 0.8951
Epoch 10/400
3/3 [==============================] - 0s 7ms/step - loss: 1.2417
Epoch 11/400
3/3 [==============================] - 0s 6ms/step - loss: 0.6540
Epoch 12/400
3/3 [==============================] - 0s 9ms/step - loss: 0.6288
Epoch 13/400
3/3 [==============================] - 0s 10m

3/3 [==============================] - 0s 7ms/step - loss: 0.0116
Epoch 105/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0116
Epoch 106/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0105
Epoch 107/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0121
Epoch 108/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0110
Epoch 109/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0155
Epoch 110/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0104
Epoch 111/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0105
Epoch 112/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0094
Epoch 113/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0100
Epoch 114/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0107
Epoch 115/400
3/3 [==============================] - 0s 8ms/step - loss: 0.0108
Epoch 116/400
3/3 [==============================] - 0

3/3 [==============================] - 0s 7ms/step - loss: 0.0055
Epoch 207/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0047
Epoch 208/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0055
Epoch 209/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0089
Epoch 210/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0049
Epoch 211/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0084
Epoch 212/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0095
Epoch 213/400
3/3 [==============================] - 0s 9ms/step - loss: 0.0196
Epoch 214/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0151
Epoch 215/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0173
Epoch 216/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0101
Epoch 217/400
3/3 [==============================] - 0s 10ms/step - loss: 0.0091
Epoch 218/400
3/3 [==============================] - 

3/3 [==============================] - 0s 6ms/step - loss: 0.0397
Epoch 309/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0195
Epoch 310/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0296
Epoch 311/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0219
Epoch 312/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0179
Epoch 313/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0563
Epoch 314/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0311
Epoch 315/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0113
Epoch 316/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0288
Epoch 317/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0361
Epoch 318/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0525
Epoch 319/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0597
Epoch 320/400
3/3 [==============================] - 0

{'Target Variable': 'Avg_Soil_TP100_TempC_Plus_1',
 'Best Parameters': {'activation': 'relu',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 6,
  'neurons': 350,
  'optimizer': 'Adam'},
 'R Squared': 0.9995060229302215,
 'MAE': 0.0740674382226467,
 'MSE': 0.011275362770768666}

### Max_Soil_TP100_TempC_Plus_1

In [73]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [350],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [74]:
i = "Max_Soil_TP100_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
3/3 [==============================] - 0s 5ms/step - loss: 59.5857
Epoch 2/400
3/3 [==============================] - 0s 5ms/step - loss: 28.6485
Epoch 3/400
3/3 [==============================] - 0s 4ms/step - loss: 25.9694
Epoch 4/400
3/3 [==============================] - 0s 4ms/step - loss: 29.3168
Epoch 5/400
3/3 [==============================] - 0s 5ms/step - loss: 28.7996
Epoch 6/400
3/3 [==============================] - 0s 5ms/step - loss: 26.2249
Epoch 7/400
3/3 [==============================] - 0s 6ms/step - loss: 24.6079
Epoch 8/400
3/3 [==============================] - 0s 4ms/step - loss: 24.3427
Epoch 9/400
3/3 [==============================] - 0s 5ms/step - loss: 24.6754
Epoch 10/400
3/3 [==============================] - 0s 4ms/step - loss: 24.6274
Epoch 11/400
3/3 [==============================] - 0s 4ms/step - loss: 24.1911
Epoch 12/400
3/3 [==============================] - 0s 4ms/step - loss: 23.7408
Epoch 13/400
3/3 [==============================]

3/3 [==============================] - 0s 4ms/step - loss: 0.3228
Epoch 105/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3197
Epoch 106/400
3/3 [==============================] - 0s 4ms/step - loss: 0.3182
Epoch 107/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3253
Epoch 108/400
3/3 [==============================] - 0s 4ms/step - loss: 0.3178
Epoch 109/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3229
Epoch 110/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3195
Epoch 111/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3240
Epoch 112/400
3/3 [==============================] - 0s 4ms/step - loss: 0.3371
Epoch 113/400
3/3 [==============================] - 0s 4ms/step - loss: 0.3431
Epoch 114/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3094
Epoch 115/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3545
Epoch 116/400
3/3 [==============================] - 0

Epoch 206/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1441
Epoch 207/400
3/3 [==============================] - 0s 4ms/step - loss: 0.1321
Epoch 208/400
3/3 [==============================] - 0s 4ms/step - loss: 0.1193
Epoch 209/400
3/3 [==============================] - 0s 4ms/step - loss: 0.1243
Epoch 210/400
3/3 [==============================] - 0s 4ms/step - loss: 0.1279
Epoch 211/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1552
Epoch 212/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1618
Epoch 213/400
3/3 [==============================] - 0s 6ms/step - loss: 0.1247
Epoch 214/400
3/3 [==============================] - 0s 4ms/step - loss: 0.1200
Epoch 215/400
3/3 [==============================] - 0s 4ms/step - loss: 0.1342
Epoch 216/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1398
Epoch 217/400
3/3 [==============================] - 0s 4ms/step - loss: 0.1325
Epoch 218/400
3/3 [=====================

3/3 [==============================] - 0s 5ms/step - loss: 0.0524
Epoch 309/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0595
Epoch 310/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0536
Epoch 311/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0489
Epoch 312/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0477
Epoch 313/400
3/3 [==============================] - ETA: 0s - loss: 0.055 - 0s 5ms/step - loss: 0.0545
Epoch 314/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0570
Epoch 315/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0488
Epoch 316/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0482
Epoch 317/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0479
Epoch 318/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0478
Epoch 319/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0449
Epoch 320/400
3/3 [===========

{'Target Variable': 'Max_Soil_TP100_TempC_Plus_1',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 350,
  'optimizer': 'Adam'},
 'R Squared': 0.9979091168265113,
 'MAE': 0.1901383701457978,
 'MSE': 0.06752478541978892}

### Min_Soil_TP100_TempC_Plus_1

In [75]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [150],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [76]:
i = "Min_Soil_TP100_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
3/3 [==============================] - 0s 3ms/step - loss: 61.5149
Epoch 2/400
3/3 [==============================] - 0s 2ms/step - loss: 47.8337
Epoch 3/400
3/3 [==============================] - 0s 3ms/step - loss: 37.9286
Epoch 4/400
3/3 [==============================] - 0s 3ms/step - loss: 30.6829
Epoch 5/400
3/3 [==============================] - 0s 2ms/step - loss: 26.7153
Epoch 6/400
3/3 [==============================] - 0s 4ms/step - loss: 24.6754
Epoch 7/400
3/3 [==============================] - 0s 3ms/step - loss: 24.1576
Epoch 8/400
3/3 [==============================] - 0s 4ms/step - loss: 24.3867
Epoch 9/400
3/3 [==============================] - 0s 3ms/step - loss: 24.6465
Epoch 10/400
3/3 [==============================] - 0s 3ms/step - loss: 24.8660
Epoch 11/400
3/3 [==============================] - 0s 2ms/step - loss: 24.8411
Epoch 12/400
3/3 [==============================] - 0s 3ms/step - loss: 24.6659
Epoch 13/400
3/3 [==============================]

3/3 [==============================] - 0s 3ms/step - loss: 0.3621
Epoch 105/400
3/3 [==============================] - 0s 4ms/step - loss: 0.3615
Epoch 106/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3603
Epoch 107/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3589
Epoch 108/400
3/3 [==============================] - 0s 2ms/step - loss: 0.3611
Epoch 109/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3571
Epoch 110/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3562
Epoch 111/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3597
Epoch 112/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3549
Epoch 113/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3577
Epoch 114/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3537
Epoch 115/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3534
Epoch 116/400
3/3 [==============================] - 0

3/3 [==============================] - 0s 3ms/step - loss: 0.1986
Epoch 205/400
3/3 [==============================] - 0s 3ms/step - loss: 0.2182
Epoch 206/400
3/3 [==============================] - 0s 2ms/step - loss: 0.2385
Epoch 207/400
3/3 [==============================] - 0s 3ms/step - loss: 0.1870
Epoch 208/400
3/3 [==============================] - 0s 3ms/step - loss: 0.1634
Epoch 209/400
3/3 [==============================] - 0s 2ms/step - loss: 0.1653
Epoch 210/400
3/3 [==============================] - 0s 3ms/step - loss: 0.1694
Epoch 211/400
3/3 [==============================] - 0s 3ms/step - loss: 0.1847
Epoch 212/400
3/3 [==============================] - 0s 2ms/step - loss: 0.1442
Epoch 213/400
3/3 [==============================] - 0s 3ms/step - loss: 0.1635
Epoch 214/400
3/3 [==============================] - 0s 3ms/step - loss: 0.1715
Epoch 215/400
3/3 [==============================] - 0s 2ms/step - loss: 0.1289
Epoch 216/400
3/3 [==============================] - 0

Epoch 306/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0345
Epoch 307/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0348
Epoch 308/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0416
Epoch 309/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0449
Epoch 310/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0475
Epoch 311/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0530
Epoch 312/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0487
Epoch 313/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0533
Epoch 314/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0322
Epoch 315/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0286
Epoch 316/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0348
Epoch 317/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0291
Epoch 318/400
3/3 [=====================

{'Target Variable': 'Min_Soil_TP100_TempC_Plus_1',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 150,
  'optimizer': 'Adam'},
 'R Squared': 0.9986463508467999,
 'MAE': 0.14374471510958667,
 'MSE': 0.042598264077079034}

### Avg_Soil_TP100_TempC_Plus_2

In [77]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [350],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [78]:
i = "Avg_Soil_TP100_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
3/3 [==============================] - 0s 6ms/step - loss: 47.5910
Epoch 2/400
3/3 [==============================] - 0s 4ms/step - loss: 25.5137
Epoch 3/400
3/3 [==============================] - 0s 5ms/step - loss: 27.3168
Epoch 4/400
3/3 [==============================] - 0s 8ms/step - loss: 28.8884
Epoch 5/400
3/3 [==============================] - 0s 4ms/step - loss: 26.3756
Epoch 6/400
3/3 [==============================] - 0s 5ms/step - loss: 24.8397
Epoch 7/400
3/3 [==============================] - 0s 5ms/step - loss: 24.6635
Epoch 8/400
3/3 [==============================] - 0s 5ms/step - loss: 25.1638
Epoch 9/400
3/3 [==============================] - 0s 4ms/step - loss: 24.7248
Epoch 10/400
3/3 [==============================] - 0s 7ms/step - loss: 23.9872
Epoch 11/400
3/3 [==============================] - 0s 4ms/step - loss: 23.6953
Epoch 12/400
3/3 [==============================] - 0s 4ms/step - loss: 23.4260
Epoch 13/400
3/3 [==============================]

3/3 [==============================] - 0s 4ms/step - loss: 0.3126
Epoch 105/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3148
Epoch 106/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3229
Epoch 107/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3239
Epoch 108/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3266
Epoch 109/400
3/3 [==============================] - 0s 4ms/step - loss: 0.3065
Epoch 110/400
3/3 [==============================] - 0s 4ms/step - loss: 0.3101
Epoch 111/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3325
Epoch 112/400
3/3 [==============================] - 0s 4ms/step - loss: 0.2992
Epoch 113/400
3/3 [==============================] - 0s 4ms/step - loss: 0.2953
Epoch 114/400
3/3 [==============================] - 0s 5ms/step - loss: 0.3062
Epoch 115/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2942
Epoch 116/400
3/3 [==============================] - 0

3/3 [==============================] - 0s 5ms/step - loss: 0.1739
Epoch 207/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1396
Epoch 208/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1607
Epoch 209/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1414
Epoch 210/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1614
Epoch 211/400
3/3 [==============================] - 0s 4ms/step - loss: 0.1660
Epoch 212/400
3/3 [==============================] - 0s 6ms/step - loss: 0.1583
Epoch 213/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1329
Epoch 214/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1319
Epoch 215/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1276
Epoch 216/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1239
Epoch 217/400
3/3 [==============================] - 0s 4ms/step - loss: 0.1382
Epoch 218/400
3/3 [==============================] - 0

3/3 [==============================] - 0s 6ms/step - loss: 0.0749
Epoch 309/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0739
Epoch 310/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0686
Epoch 311/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0778
Epoch 312/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0616
Epoch 313/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0678
Epoch 314/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0669
Epoch 315/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0589
Epoch 316/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0639
Epoch 317/400
3/3 [==============================] - 0s 7ms/step - loss: 0.0623
Epoch 318/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0637
Epoch 319/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0615
Epoch 320/400
3/3 [==============================] - 0

{'Target Variable': 'Avg_Soil_TP100_TempC_Plus_2',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 350,
  'optimizer': 'Adam'},
 'R Squared': 0.9973510824861164,
 'MAE': 0.18145341611766816,
 'MSE': 0.06371578524229224}

### Max_Soil_TP100_TempC_Plus_2

In [79]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [350],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [80]:
i = "Max_Soil_TP100_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
3/3 [==============================] - 0s 5ms/step - loss: 43.9918
Epoch 2/400
3/3 [==============================] - 0s 5ms/step - loss: 25.0182
Epoch 3/400
3/3 [==============================] - 0s 5ms/step - loss: 27.3912
Epoch 4/400
3/3 [==============================] - 0s 8ms/step - loss: 27.6970
Epoch 5/400
3/3 [==============================] - 0s 6ms/step - loss: 25.4607
Epoch 6/400
3/3 [==============================] - 0s 5ms/step - loss: 24.3438
Epoch 7/400
3/3 [==============================] - 0s 5ms/step - loss: 24.6256
Epoch 8/400
3/3 [==============================] - 0s 5ms/step - loss: 24.5864
Epoch 9/400
3/3 [==============================] - 0s 5ms/step - loss: 24.0698
Epoch 10/400
3/3 [==============================] - 0s 5ms/step - loss: 23.4325
Epoch 11/400
3/3 [==============================] - 0s 4ms/step - loss: 22.6691
Epoch 12/400
3/3 [==============================] - 0s 4ms/step - loss: 21.3671
Epoch 13/400
3/3 [==============================]

3/3 [==============================] - 0s 4ms/step - loss: 0.2886
Epoch 105/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2930
Epoch 106/400
3/3 [==============================] - 0s 4ms/step - loss: 0.2850
Epoch 107/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2808
Epoch 108/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2863
Epoch 109/400
3/3 [==============================] - 0s 6ms/step - loss: 0.2828
Epoch 110/400
3/3 [==============================] - 0s 4ms/step - loss: 0.2823
Epoch 111/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2764
Epoch 112/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2905
Epoch 113/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2854
Epoch 114/400
3/3 [==============================] - 0s 6ms/step - loss: 0.2812
Epoch 115/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2849
Epoch 116/400
3/3 [==============================] - 0

3/3 [==============================] - 0s 5ms/step - loss: 0.2090
Epoch 207/400
3/3 [==============================] - 0s 6ms/step - loss: 0.1673
Epoch 208/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2899
Epoch 209/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1895
Epoch 210/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2097
Epoch 211/400
3/3 [==============================] - 0s 5ms/step - loss: 0.2678
Epoch 212/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1819
Epoch 213/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1311
Epoch 214/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1267
Epoch 215/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1494
Epoch 216/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1344
Epoch 217/400
3/3 [==============================] - 0s 6ms/step - loss: 0.1265
Epoch 218/400
3/3 [==============================] - 0

3/3 [==============================] - 0s 5ms/step - loss: 0.0934
Epoch 309/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1222
Epoch 310/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0878
Epoch 311/400
3/3 [==============================] - 0s 5ms/step - loss: 0.1019
Epoch 312/400
3/3 [==============================] - 0s 6ms/step - loss: 0.0972
Epoch 313/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0859
Epoch 314/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0861
Epoch 315/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0781
Epoch 316/400
3/3 [==============================] - 0s 4ms/step - loss: 0.0825
Epoch 317/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0769
Epoch 318/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0848
Epoch 319/400
3/3 [==============================] - 0s 5ms/step - loss: 0.0716
Epoch 320/400
3/3 [==============================] - 0

{'Target Variable': 'Max_Soil_TP100_TempC_Plus_2',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 350,
  'optimizer': 'Adam'},
 'R Squared': 0.9975607496656249,
 'MAE': 0.16719204335880275,
 'MSE': 0.05387344939945391}

### Min_Soil_TP100_TempC_Plus_2

In [81]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [150],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [82]:
i = "Min_Soil_TP100_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
3/3 [==============================] - 0s 3ms/step - loss: 49.6314
Epoch 2/400
3/3 [==============================] - 0s 3ms/step - loss: 38.6125
Epoch 3/400
3/3 [==============================] - 0s 3ms/step - loss: 30.9543
Epoch 4/400
3/3 [==============================] - 0s 2ms/step - loss: 26.1893
Epoch 5/400
3/3 [==============================] - 0s 3ms/step - loss: 24.4760
Epoch 6/400
3/3 [==============================] - 0s 4ms/step - loss: 24.3826
Epoch 7/400
3/3 [==============================] - 0s 2ms/step - loss: 24.8166
Epoch 8/400
3/3 [==============================] - 0s 2ms/step - loss: 25.2135
Epoch 9/400
3/3 [==============================] - 0s 3ms/step - loss: 25.2682
Epoch 10/400
3/3 [==============================] - 0s 4ms/step - loss: 24.9627
Epoch 11/400
3/3 [==============================] - 0s 2ms/step - loss: 24.4524
Epoch 12/400
3/3 [==============================] - 0s 3ms/step - loss: 23.9176
Epoch 13/400
3/3 [==============================]

3/3 [==============================] - 0s 3ms/step - loss: 0.3204
Epoch 104/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3268
Epoch 105/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3211
Epoch 106/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3165
Epoch 107/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3234
Epoch 108/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3201
Epoch 109/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3172
Epoch 110/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3249
Epoch 111/400
3/3 [==============================] - 0s 2ms/step - loss: 0.3202
Epoch 112/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3192
Epoch 113/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3189
Epoch 114/400
3/3 [==============================] - 0s 3ms/step - loss: 0.3155
Epoch 115/400
3/3 [==============================] - 0

3/3 [==============================] - 0s 3ms/step - loss: 0.0958
Epoch 206/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0925
Epoch 207/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0863
Epoch 208/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0862
Epoch 209/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0903
Epoch 210/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0886
Epoch 211/400
3/3 [==============================] - 0s 3ms/step - loss: 0.1044
Epoch 212/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0973
Epoch 213/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0994
Epoch 214/400
3/3 [==============================] - 0s 3ms/step - loss: 0.1125
Epoch 215/400
3/3 [==============================] - 0s 3ms/step - loss: 0.1308
Epoch 216/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0768
Epoch 217/400
3/3 [==============================] - 0

Epoch 307/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0303
Epoch 308/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0269
Epoch 309/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0277
Epoch 310/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0255
Epoch 311/400
3/3 [==============================] - ETA: 0s - loss: 0.023 - 0s 3ms/step - loss: 0.0286
Epoch 312/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0260
Epoch 313/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0257
Epoch 314/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0270
Epoch 315/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0245
Epoch 316/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0246
Epoch 317/400
3/3 [==============================] - 0s 2ms/step - loss: 0.0244
Epoch 318/400
3/3 [==============================] - 0s 3ms/step - loss: 0.0295
Epoch 319/400
3/

{'Target Variable': 'Min_Soil_TP100_TempC_Plus_2',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 150,
  'optimizer': 'Adam'},
 'R Squared': 0.9987200066707683,
 'MAE': 0.11854698974943162,
 'MSE': 0.030697584262968626}